In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os, sys

In [2]:
current_dir=os.getcwd()                     #current working directory
print(current_dir)
parent_dir=os.path.dirname(current_dir)     # Get the parent directory
print(parent_dir)
sys.path.append(parent_dir)

c:\10acadamy\week2\TellCo-Analysis\notebooks
c:\10acadamy\week2\TellCo-Analysis


In [3]:
from scripts.general_purpose import fetch_data_as_dataframe
from scripts.general_purpose import explore_dataframe

In [4]:
# Get data from PostgreSQL
# SQL query to fetch data
query = "SELECT * FROM xdr_data;"
df_tellco=fetch_data_as_dataframe(query)

In [ ]:
# print dataframe columns
explore_dataframe(df_tellco)


<bound method NDFrame.describe of            Bearer Id            Start  Start ms              End  End ms  \
0       1.311448e+19   4/4/2019 12:01     770.0  4/25/2019 14:35   662.0   
1       1.311448e+19   4/9/2019 13:04     235.0   4/25/2019 8:15   606.0   
2       1.311448e+19   4/9/2019 17:42       1.0  4/25/2019 11:58   652.0   
3       1.311448e+19   4/10/2019 0:31     486.0   4/25/2019 7:36   171.0   
4       1.311448e+19  4/12/2019 20:10     565.0  4/25/2019 10:40   954.0   
...              ...              ...       ...              ...     ...   
149996  7.277826e+18   4/29/2019 7:28     451.0   4/30/2019 6:02   214.0   
149997  7.349883e+18   4/29/2019 7:28     483.0  4/30/2019 10:41   187.0   
149998  1.311448e+19   4/29/2019 7:28     283.0  4/30/2019 10:46   810.0   
149999  1.311448e+19   4/29/2019 7:28     696.0  4/30/2019 10:40   327.0   
150000           NaN             None       NaN             None     NaN   

        Dur. (ms)          IMSI  MSISDN/Number       

In [ ]:
top_handsets = (
    df_tellco.groupby('Handset Type')
)

print(top_handsets.head())

           Bearer Id            Start  Start ms              End  End ms  \
0       1.311448e+19   4/4/2019 12:01     770.0  4/25/2019 14:35   662.0   
1       1.311448e+19   4/9/2019 13:04     235.0   4/25/2019 8:15   606.0   
2       1.311448e+19   4/9/2019 17:42       1.0  4/25/2019 11:58   652.0   
3       1.311448e+19   4/10/2019 0:31     486.0   4/25/2019 7:36   171.0   
4       1.311448e+19  4/12/2019 20:10     565.0  4/25/2019 10:40   954.0   
...              ...              ...       ...              ...     ...   
149489  1.304243e+19   4/29/2019 7:23     421.0   4/30/2019 7:23   395.0   
149599  6.917538e+18   4/29/2019 7:24     750.0   4/30/2019 7:24   375.0   
149606  6.917538e+18   4/29/2019 7:24     632.0   4/30/2019 7:01   120.0   
149744  6.917538e+18   4/29/2019 7:26     679.0   4/30/2019 7:16   474.0   
149860  6.917538e+18   4/29/2019 7:27     328.0   4/30/2019 7:29   168.0   

        Dur. (ms)          IMSI  MSISDN/Number          IMEI  \
0       1823652.0  2.08